In [1]:
import tensorflow
import keras
from keras.models import model_from_json
import matplotlib.pyplot as plt
import pandas as pd
import os
import idlsave
import numpy as np
from tqdm import tqdm_notebook as tqdm
import re
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
import glob
import seaborn as sns

Using TensorFlow backend.


In [2]:
model = model_from_json(open('model.json', 'rb').read())

In [3]:
root_path = '/Users/localhost/Desktop/Projects/Working/CFA/visioneering-deeplearning/experiments/planet_experiments/local_cache/original_k2_data/'
data_path = root_path+'curves/'
weights_path = 'weights/'
label_path = root_path+'labels/joined_labels.csv'
validation_campaigns = root_path+'Validation_Campaigns.csv'
example_weights = 'weights/lstm_reattempt_weights.05-0.77.hdf5'
roc_curves = 'out/'

validation_campaigns = pd.read_csv(validation_campaigns)
validation_campaigns = pd.DataFrame({0: ['c0']})
labels = pd.read_csv(label_path)
del labels['junk']
id_from_filepath = re.compile('(?<=ep)[0-9]*(?=search)')

In [4]:
def plot_roc(tpr,fpr,roc_auc,label, saving_path):
    if not os.path.exists(os.path.join(saving_path,'roc_curves')):
        os.makedirs(os.path.join(saving_path,'roc_curves'))
    plt.figure()    
    plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='R OC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.savefig(saving_path + 'roc_curves/' + label + '.png')

In [5]:
def get_n_samples(n):
    size = 1360
    errors = 0
    for campaign in validation_campaigns[0]:
        files = [data_path+campaign+'/'+x for x in np.array(os.listdir(data_path+campaign))]
        for fs in np.array_split(files, int(len(files)/n)):
            X = []
            y = []
            for f in fs:
                try:
                    x = idlsave.read(f, verbose=False).k.f[0][:1360]
                    assert len(x) == 1360
                    p_id = id_from_filepath.findall(f)[0]
                    label = labels[labels['planet_id'] == int(p_id)]['label'].tolist()[0].strip()
                    y.append([label == 'C', label != 'C'])
                    X.append(x.reshape((-1,1)))
                except Exception as e:
                    errors += 1
            yield np.array(X), np.array(y).astype(float)

In [6]:
n_classes = 2
def get_roc(predicted, actual):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(actual[:, i], predicted[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(actual.ravel(), predicted.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    return tpr, fpr, roc_auc

In [7]:
def plot_lc(tpr, fpr, roc_auc, title):
    # Compute macro-average ROC curve and ROC area
    lw = 2
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    fig = plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    fig.savefig(roc_curves+title+'.png')

In [12]:
for f in tqdm(os.listdir(weights_path)):
    model.load_weights(weights_path+f)
    predicted = []
    actual = []
    batch_size = 1000
    # Compute predicted in batches
    for X, y in tqdm(get_n_samples(batch_size), total=int(len(glob.glob(data_path+'*/*'))/batch_size), leave=False):
        predictions = model.predict(X)
        predicted.extend(predictions)
        actual.extend(y)
    tpr, fpr, roc_auc = get_roc(np.array(predicted), np.array(actual))
    plot_lc(tpr, fpr, roc_auc, f)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [10]:
predicted

[array([ 0.47470847,  0.5252915 ], dtype=float32),
 array([ 0.47498423,  0.52501571], dtype=float32),
 array([ 0.47490191,  0.52509809], dtype=float32),
 array([ 0.47433716,  0.52566278], dtype=float32),
 array([ 0.47494712,  0.52505285], dtype=float32),
 array([ 0.47499216,  0.52500784], dtype=float32),
 array([ 0.47492382,  0.52507615], dtype=float32),
 array([ 0.47471887,  0.52528119], dtype=float32),
 array([ 0.4743996 ,  0.52560043], dtype=float32),
 array([ 0.46781263,  0.53218734], dtype=float32),
 array([ 0.47500026,  0.52499968], dtype=float32),
 array([ 0.47480476,  0.5251953 ], dtype=float32),
 array([ 0.47469011,  0.52530992], dtype=float32),
 array([ 0.47429401,  0.52570599], dtype=float32),
 array([ 0.47490942,  0.52509052], dtype=float32),
 array([ 0.47462305,  0.52537692], dtype=float32),
 array([ 0.47495481,  0.52504528], dtype=float32),
 array([ 0.47452962,  0.52547044], dtype=float32),
 array([ 0.47498402,  0.52501601], dtype=float32),
 array([ 0.47427309,  0.5257269

In [21]:
[x.argmax() for x in (np.array(predicted)>.5)]

False

In [19]:
predictions

array([[ 0.50407392,  0.49592608],
       [ 0.50407028,  0.49592966],
       [ 0.50406283,  0.49593714],
       [ 0.5040732 ,  0.49592674],
       [ 0.50436753,  0.49563256],
       [ 0.50406301,  0.49593693],
       [ 0.50406873,  0.49593127],
       [ 0.50408345,  0.49591655],
       [ 0.50409758,  0.49590248],
       [ 0.50408041,  0.49591953],
       [ 0.50405908,  0.49594089],
       [ 0.50407785,  0.49592215],
       [ 0.50415111,  0.49584886],
       [ 0.50410551,  0.49589443],
       [ 0.50420302,  0.49579704],
       [ 0.50406331,  0.49593678],
       [ 0.50411808,  0.49588192],
       [ 0.50408775,  0.49591225],
       [ 0.50408483,  0.49591517],
       [ 0.5042156 ,  0.49578443],
       [ 0.50406957,  0.49593043],
       [ 0.50405091,  0.49594912],
       [ 0.5040437 ,  0.49595633],
       [ 0.50412095,  0.49587899],
       [ 0.5039019 ,  0.4960981 ],
       [ 0.50410432,  0.49589571],
       [ 0.50405812,  0.49594182],
       [ 0.50403553,  0.4959645 ],
       [ 0.50403583,